## Classificação dos dados
### Classificação supervisionada

In [3]:
dfa = df_arruma

NameError: name 'df_arruma' is not defined

Nosso target é a largura do bico (bill length) dos pinguins, e, visto que é um target numérico, o agruparemos em 5 grupos:\
#### Grupos
1. (Muito pequeno) 0.0 até 0.2 \
dfa["bill_length_norm"] <= 0.2
2. (Pequeno) 0.2 até 0.4 \
(dfa["bill_length_norm"] > 0.2) & (dfa["bill_length_norm"] <= 0.4)\
E assim por diante
3. (Médio) 0.4 até 0.6
4. (Grande) 0.6 até 0.8
5. (Muito grande) 0.8 até 1

Na célula seguinte, organizei o valor da largura dos bicos (normalizado) em ordem crescente.

In [ ]:
dfa.sort_values(by=['bill_length_norm'])

A seguir, aplicamos o código para classificar os bicos dos pinguins.

In [ ]:
dfa["Largura do bico (categórico)"] = None

divisao = dfa["bill_length_norm"] <= 0.2
dfa.loc[divisao,"Largura do bico (categórico)"] = "Muito pequeno"
    
divisao = (dfa["bill_length_norm"] > 0.2) & (dfa["bill_length_norm"] <= 0.4) # Pequeno
dfa.loc[divisao,"Largura do bico (categórico)"] = "Pequeno"
    
divisao = (dfa["bill_length_norm"] > 0.4) & (dfa["bill_length_norm"] <= 0.6) # Médio
dfa.loc[divisao,"Largura do bico (categórico)"] = "Médio"
    
divisao = (dfa["bill_length_norm"] > 0.6) & (dfa["bill_length_norm"] <= 0.8) # Grande
dfa.loc[divisao,"Largura do bico (categórico)"] = "Grande"

divisao = (dfa["bill_length_norm"] > 0.8) & (dfa["bill_length_norm"] <= 1) # Muito grande
dfa.loc[divisao,"Largura do bico (categórico)"] = "Muito grande"

print(dfa)
print("Deu certo, apenas faltaria organizar a coluna em ordem crescente!")

#### Classificação com k-NN

Agora, vamos começar a testar os três modelos passados em aula. O primeiro sendo o k-NN, ou nearest neighbors. 

In [ ]:
from sklearn.model_selection import train_test_split

tamanho_teste = 0.1 
semente = 61454
#Definimos o tamanho do nosso teste e uma semente escolhida

features = ["body_mass_norm", "bill_depth_norm", "flipper_length_norm"] 
target = ["Largura do bico (categórico)"]
#Após isso, definimos as features que utilizaremos (neste caso todas as numéricas) nos nossos treinos e previsões
# para chegarmos no target.

In [ ]:
indices = dfa.index
indices_treino, indices_teste = train_test_split(indices, test_size = tamanho_teste, random_state = semente)
#Definimos os indices que utilizaremos (do nosso data set). Com todos os indices do data set, escolhemos
# quais usaremos para treino e para teste.


dfa_treino = dfa.loc[indices_treino]
dfa_teste = dfa.loc[indices_teste]

x_treino = dfa_treino.reindex(features, axis=1).values
x_teste = dfa_teste.reindex(features, axis=1).values

y_treino = dfa_treino.reindex(target, axis=1).values.ravel()
y_teste = dfa_teste.reindex(target, axis=1).values.ravel()


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
import numpy

modelo_vizinhos = KNeighborsClassifier()
#Após termos importado o classificador, criamos o nosso modelo


modelo_vizinhos.fit(x_treino,y_treino)
# Então treinamos o modelo com nossas variáveis de treino, através do módulo .fit


previsao1 = modelo_vizinhos.predict(x_teste)
print(previsao1)

E, com isso, temos a nossa primeira previsão! Mas ela apenas utilizou dados de teste. A seguir, vamos utilizar os dados "verdadeiros". 

In [ ]:
from sklearn import metrics

print("Precisão:", metrics.accuracy_score(y_teste, previsao1))

#### Classificação com árvore de decisão

In [ ]:
from sklearn.tree import DecisionTreeClassifier

modelo_arvore = DecisionTreeClassifier(random_state = semente)

modelo_arvore.fit(x_treino, y_treino)

previsao2 = modelo_arvore.predict(x_teste)
print(previsao2)

In [ ]:
print("Precisão:", metrics.accuracy_score(y_teste, previsao2))

#### Classificação com florestas aleatórias

In [ ]:
from sklearn.ensemble import RandomForestClassifier

modelo_floresta = RandomForestClassifier(random_state = semente)

modelo_floresta.fit(x_treino, y_treino)

previsao3 = modelo_arvore.predict(x_teste)
print(previsao3)

In [ ]:
print("Precisão:", metrics.accuracy_score(y_teste, previsao3))

As hipóteses para essas baixas precisões/accuracy são: o baixo número de parametros, a utilização APENAS de parametros numéricos e a não utilização de hiperparametros.

#### Tentativa de refinar os dados com hiperparâmetros

Assim como sugerido no final do bloco da última aula, tentei refinar os dados com hiperparametros utilizando o grid search. E, mesmo que tenha ajudado, creio que ainda seja necessário aumentar os parametros a serem utilizados.

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [200, 300, 400, 500, 600],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,6,8,10,12,14,16],
    'criterion' :['gini', 'entropy']
}

CV_rfc = GridSearchCV(estimator=modelo_floresta, param_grid=param_grid, cv= 5, n_jobs=8)
CV_rfc.fit(x_treino, y_treino)

In [ ]:
print(CV_rfc.best_params_)
best_params = CV_rfc.best_params_

In [ ]:
best_params = {'criterion': 'entropy', 'max_depth': 14, 'max_features': 'sqrt', 'n_estimators': 600}

modelo_floresta = RandomForestClassifier(random_state=semente)
modelo_floresta.set_params(**best_params)


In [1]:
modelo_floresta.fit(x_treino, y_treino)
previsao = modelo_floresta.predict(x_teste)
print(previsao)
print("Accuracy:",metrics.accuracy_score(y_teste, previsao))
print()
print("Melhorou bem pouco.")

NameError: name 'modelo_floresta' is not defined

Correção 24/10/2022:

Agora nosso target é a espécie do pinguim, o que está sendo trabalhado nos próximos blocos. 